# Compute i coefficients

In [118]:
import brightway2 as bw
import pandas as pd
import os
import numpy as np

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

#Constant
drilling_waste_per_metre = 450

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

In [119]:
coeff_matrix


,i1,i2.1,i2.2,i2.3,i2.4,i2.5,i2.6,i3,i4.1,i4.2,i4.3,i5.1,i5.2,i6
"(ILCD 2.0 2018 midpoint no LT, climate change, climate change total)",41951.729340,8.777813e-02,1.924688e+00,8.724747e-01,4.170591e+01,7.221693e-03,1.619566e+00,618.833223,1.363071e+05,2.901737e+05,1.933752e+01,1.099648e-03,9.330638e-01,1.0
"(ILCD 2.0 2018 midpoint no LT, human health, carcinogenic effects)",0.009333,1.338255e-10,6.007013e-07,3.485216e-09,5.328093e-07,9.399042e-08,9.236655e-09,0.000125,1.674126e-02,1.206404e-02,2.081519e-07,8.015878e-11,4.193463e-09,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, ionising radiation)",1130.168402,5.180734e-03,5.165839e-02,1.276488e-02,8.931351e-01,6.417186e-04,3.551923e-02,15.724582,4.156112e+03,2.173757e+04,2.324622e-01,7.549285e-05,5.396868e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, non-carcinogenic effects)",0.022149,1.937160e-09,1.379242e-06,4.700238e-08,5.492678e-06,2.914533e-07,1.095188e-07,0.000300,6.103437e-02,1.316675e-01,1.493808e-06,3.343961e-10,3.053899e-08,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, ozone layer depletion)",0.002771,1.917973e-08,1.135916e-07,3.159978e-08,4.214417e-06,2.230813e-09,8.984558e-08,0.000039,1.270293e-02,1.479841e-01,7.843954e-04,7.157074e-11,1.985066e-07,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, photochemical ozone creation)",198.841132,1.590873e-03,9.231551e-03,1.970287e-03,1.899794e-01,6.004382e-05,4.846082e-03,2.818856,5.916199e+02,8.933771e+02,4.296965e-02,3.300982e-06,1.643018e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, respiratory effects, inorganics)",0.003734,1.572051e-09,1.752617e-07,1.961009e-08,2.025714e-06,7.918533e-10,5.230986e-08,0.000051,9.882556e-03,1.355099e-02,6.220811e-07,5.597777e-11,1.844747e-08,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater and terrestrial acidification)",225.039103,1.242693e-03,9.812659e-03,2.418912e-03,1.981686e-01,5.601000e-05,5.682148e-03,3.459902,9.570330e+02,1.766334e+03,6.213826e-02,5.628260e-06,1.296500e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater ecotoxicity)",118226.208018,1.458437e-02,7.242976e+00,1.629244e-01,2.966899e+01,5.239158e+00,6.021999e-01,1587.931584,2.904214e+05,3.394154e+05,8.931917e+00,1.298698e-03,2.030111e-01,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater eutrophication)",1.973278,1.373653e-07,1.131584e-04,1.085733e-05,2.125317e-03,1.518214e-05,2.468011e-05,0.028690,7.573346e+00,3.852353e+01,2.961919e-04,6.684274e-08,3.243362e-06,0.0


# Enhanced geothermal

# Define symbolic

In [226]:
from sympy import symbols, collect, ratsimp, fraction, factor_terms

In [227]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Main parameters
W_Pn, P_ne , PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP, SW_n, S_w, S_el  = \
symbols('W_Pn, P_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP, SW_n, SW, S_el')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
E_co2 = symbols('E_co2')

# Constants
CT_el, CT_n, W_en, OF, DW = symbols ('CT_el, CT_n, W_E_en, OF, DW')


# Main equation


In [228]:
# Number of wells
W_n = W_Pn
W_E_en = W_en

# With success rate
W_n_sr = W_n/(SR_p/100)
W_E_en_sr = W_E_en/(SR_e/100)

In [229]:
# Equation
nominator = (W_n + W_E_en) *i1 \
          + W_d* (W_n_sr + W_E_en_sr) * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + DW*i2_5 + i2_6) + \
          + W_n*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w* (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-AP)*LT*8760000
#d1 = CT_el*CT_n*1000*LT
denominator = c1  # -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [230]:
eq

E_co2*i6 + (CP*W_Pn*i3 + P_ne*(CT_n*i4_2 + OF*i4_3 + i4_1) + SW*SW_n*(S_el*i5_2 + i5_1) + W_d*(100*W_Pn/SR_p + 100*W_E_en/SR_e)*(C_C*i2_3 + C_S*i2_2 + D*i2_1 + DM*i2_4 + DW*i2_5 + i2_6) + i1*(W_E_en + W_Pn))/(8760000*CF*LT*P_ne*(1 - AP))

# Get parameters value

In [297]:
#Options
exploration = True # For simplified models in paper
success_rate = False # For TWI software

In [298]:
# Klausen
from ege_klausen import get_parameters
parameters = get_parameters()
parameters.static()   

# Parameters of simplified models
D_val      = parameters["specific_diesel_consumption"]
W_d_val      = parameters["average_depth_of_wells"]
P_ne_val     = parameters["installed_capacity"]
if success_rate == True:
    SR_p_val     = parameters["success_rate_primary_wells"]
elif success_rate == False:
    SR_p_val     = 100

# Parameters to be fixed
LT_val     = parameters["lifetime"]
if success_rate == True:
    SR_e_val   = parameters["success_rate_exploration_wells"]
elif success_rate == False:
    SR_e_val = 100
C_S_val    = parameters["specific_steel_consumption"]
C_C_val    = parameters["specific_cement_consumption"]
DM_val     = parameters["specific_drilling_mud_consumption"]
CP_val     = parameters["collection_pipelines"]
CF_val     = parameters["capacity_factor"]
AP_val     = parameters["auxiliary_power"]
W_Pn_val   = 2.5
SW_n_val   = np.round(0.5 + parameters["number_of_wells_stimulated_0to1"] * 2.5)
S_w_val    = parameters["water_stimulation"]
S_el_val   = parameters["specific_electricity_stimulation"] / 1000

# Parameters that are not used in ENHANCED
D_i_val     = 0
E_co2_val   = 0

# Constants
CT_el_val  = 864
CT_n_val   = 7/303.3
if exploration == True:
    W_en_val   = 3 * 0.3
if exploration == False:    
    W_en_val = 0.000000000001 # This is for "no exploration" option
OF_val     = 300
DW_val     = 450

par_dict = { LT: LT_val,
             SR_e: SR_e_val,
             C_S: C_S_val,
             C_C: C_C_val,
             DM: DM_val,
             CP: CP_val,
             CF: CF_val,
             AP: AP_val,
             W_Pn: W_Pn_val,
             SW_n: SW_n_val,
             S_w: S_w_val,
             S_el: S_el_val,
             D_i: D_i_val,
             E_co2: E_co2_val, 
             CT_el: CT_el_val,
             CT_n: CT_n_val,
             W_en: W_en_val,
             OF: OF_val,
             DW: DW_val}

# Groups of environmental categories

In [299]:
# Group2 uses Diesel consumption


group1_05 = ["carcinogenic effects", "non-carcinogenic effects", "respiratory effects, inorganics",  "freshwater ecotoxicity",
             "freshwater eutrophication", "minerals and metals", "dissipated water", "land use"]  

group2_05 = ["climate change total", "ionising radiation", "ozone layer depletion", "photochemical ozone creation", \
            "freshwater and terrestrial acidification", "marine eutrophication", "terrestrial eutrophication", "fossils"]


group1_10 = ["climate change total",  "carcinogenic effects", "non-carcinogenic effects",\
              "respiratory effects, inorganics",  "freshwater ecotoxicity", "freshwater eutrophication", \
              "minerals and metals", "dissipated water", "fossils", "land use"]            

group2_10 = ["ionising radiation", "ozone layer depletion", "photochemical ozone creation", \
            "freshwater and terrestrial acidification", "marine eutrophication", "terrestrial eutrophication"]


group1_15 = ["climate change total", "carcinogenic effects", "ionising radiation", "non-carcinogenic effects", \
            "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics", \
            "freshwater and terrestrial acidification", "freshwater ecotoxicity", "freshwater eutrophication", \
            "minerals and metals", "dissipated water", "fossils", "land use"]

group2_15 = ["marine eutrophication", "terrestrial eutrophication"]

group_20 = [method[2] for method in ILCD]


# Simplified equations

In [300]:
chi_1, chi_2, chi_3, chi_4, chi_5, chi_6, chi_7, delta_1, delta_2, delta_3, delta_4, delta_5, delta_6,\
delta_7, delta_8, delta_9 = \
symbols('chi_1, chi_2, chi_3, chi_4, chi_5, chi_6, chi_7, delta_1, delta_2, delta_3, delta_4, delta_5, \
delta_6, delta_7, delta_8, delta_9')


## Threshold 20% - one group only

In [301]:
eq_group_20 = chi_1/P_ne + chi_2
eq_group_20

chi_2 + chi_1/P_ne

## Threhold 15/10% - group 1

In [302]:
eq_group1_15_10 = chi_1/P_ne + chi_2
eq_group1_15_10

chi_2 + chi_1/P_ne

## Threhold 15/10% - group 2

In [303]:
eq_group2_15_10 = (D*delta_1 + delta_2) / P_ne + delta_3
eq_group2_15_10

delta_3 + (D*delta_1 + delta_2)/P_ne

## Threshold 5% - group1

In [304]:
eq_group1_05 = (SR_p*W_d*chi_1 + SR_p*chi_2 + W_d*chi_3) / (P_ne*SR_p) + chi_4
eq_group1_05

chi_4 + (SR_p*W_d*chi_1 + SR_p*chi_2 + W_d*chi_3)/(P_ne*SR_p)

## Threshold 5% - group2

In [305]:
eq_group2_05 = (D*SR_p*W_d*delta_1 + D*W_d*delta_2 + SR_p*W_d*delta_3 + SR_p*delta_4 + W_d*delta_5) \
/ (P_ne*SR_p) + delta_6
eq_group2_05

delta_6 + (D*SR_p*W_d*delta_1 + D*W_d*delta_2 + SR_p*W_d*delta_3 + SR_p*delta_4 + W_d*delta_5)/(P_ne*SR_p)

# Compute chi

## Threshold = 20%

In [306]:
# Replace parameters in equation
eq_chi_20 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        D: D_val,
        W_d: W_d_val}
eq_chi_20 = eq_chi_20.subs(repl)

chi_dict_20 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group_20:
        eq_chi_20_is = eq_chi_20.subs(is_dict)
        eq_chi_20_is = ratsimp(eq_chi_20_is)
              
        chi1 = collect(eq_chi_20_is, 1/P_ne, evaluate=False)[1/P_ne]
        chi2 = collect(eq_chi_20_is, 1/P_ne, evaluate=False)[1]
              
        chi_dict_20[method] = {'chi1': chi1, 'chi2': chi2}
           

In [307]:
# Show example of the formula
eq_chi_20_is

4.16165746936438e-8 + 9.8427793209623e-7/P_ne

In [308]:
chi_df_20 = pd.DataFrame.from_dict(chi_dict_20, orient='index')
chi_df_20 = chi_df_20.astype(float)
chi_df_20

chi1  \
ILCD 2.0 2018 midpoint no LT climate change    climate change total                      1.078929e-01   
                             human health      carcinogenic effects                      9.040434e-09   
                                               ionising radiation                        5.504030e-03   
                                               non-carcinogenic effects                  2.547644e-08   
                                               ozone layer depletion                     1.950695e-08   
                                               photochemical ozone creation              1.597902e-03   
                                               respiratory effects, inorganics           3.428265e-09   
                             ecosystem quality freshwater and terrestrial acidification  1.285446e-03   
                                               freshwater ecotoxicity                    2.648409e-01   
                                               freshwater eutrophication                 1.936128e-06   
                                               marine eutrophication                     5.453801e-04   
                                               terrestrial eutrophication                5.963812e-03   
                             resources         dissipated water                          1.276505e-02   
                                               fossils                                   1.450794e+00   
                                               land use                                  2.017105e-01   
                                               minerals and metals                       9.842779e-07   

                                                                                                 chi2  
ILCD 2.0 2018 midpoint no LT climate change    climate change total                      7.864316e-04  
                             human health      carcinogenic effects                      9.027851e-11  
                                               ionising radiation                        2.498490e-05  
                                               non-carcinogenic effects                  3.409930e-10  
                                               ozone layer depletion                     1.328836e-09  
                                               photochemical ozone creation              3.303788e-06  
                                               respiratory effects, inorganics           5.486814e-11  
                             ecosystem quality freshwater and terrestrial acidification  5.371853e-06  
                                               freshwater ecotoxicity                    1.590428e-03  
                                               freshwater eutrophication                 4.519336e-08  
                                               marine eutrophication                     1.008642e-06  
                                               terrestrial eutrophication                9.592106e-06  
                             resources         dissipated water                          3.853415e-04  
                                               fossils                                   1.065016e-02  
                                               land use                                  5.907572e-03  
                                               minerals and metals                       4.161657e-08

## Threshold 15%

In [309]:
level_2_=chi_df_20.index.get_level_values(2)
mask=[m in group1_15 for m in level_2_ ]
chi_df_15 = chi_df_20[mask]
chi_df_15

chi1  \
ILCD 2.0 2018 midpoint no LT climate change    climate change total                      1.078929e-01   
                             human health      carcinogenic effects                      9.040434e-09   
                                               ionising radiation                        5.504030e-03   
                                               non-carcinogenic effects                  2.547644e-08   
                                               ozone layer depletion                     1.950695e-08   
                                               photochemical ozone creation              1.597902e-03   
                                               respiratory effects, inorganics           3.428265e-09   
                             ecosystem quality freshwater and terrestrial acidification  1.285446e-03   
                                               freshwater ecotoxicity                    2.648409e-01   
                                               freshwater eutrophication                 1.936128e-06   
                             resources         dissipated water                          1.276505e-02   
                                               fossils                                   1.450794e+00   
                                               land use                                  2.017105e-01   
                                               minerals and metals                       9.842779e-07   

                                                                                                 chi2  
ILCD 2.0 2018 midpoint no LT climate change    climate change total                      7.864316e-04  
                             human health      carcinogenic effects                      9.027851e-11  
                                               ionising radiation                        2.498490e-05  
                                               non-carcinogenic effects                  3.409930e-10  
                                               ozone layer depletion                     1.328836e-09  
                                               photochemical ozone creation              3.303788e-06  
                                               respiratory effects, inorganics           5.486814e-11  
                             ecosystem quality freshwater and terrestrial acidification  5.371853e-06  
                                               freshwater ecotoxicity                    1.590428e-03  
                                               freshwater eutrophication                 4.519336e-08  
                             resources         dissipated water                          3.853415e-04  
                                               fossils                                   1.065016e-02  
                                               land use                                  5.907572e-03  
                                               minerals and metals                       4.161657e-08

## Threshold 10%

In [310]:
# Select from chi_df_20 the categories that are in group1_15
level_2_=chi_df_20.index.get_level_values(2)
mask=[m in group1_10 for m in level_2_ ]
chi_df_10 = chi_df_20[mask]
chi_df_10

chi1  \
ILCD 2.0 2018 midpoint no LT climate change    climate change total             1.078929e-01   
                             human health      carcinogenic effects             9.040434e-09   
                                               non-carcinogenic effects         2.547644e-08   
                                               respiratory effects, inorganics  3.428265e-09   
                             ecosystem quality freshwater ecotoxicity           2.648409e-01   
                                               freshwater eutrophication        1.936128e-06   
                             resources         dissipated water                 1.276505e-02   
                                               fossils                          1.450794e+00   
                                               land use                         2.017105e-01   
                                               minerals and metals              9.842779e-07   

                                                                                        chi2  
ILCD 2.0 2018 midpoint no LT climate change    climate change total             7.864316e-04  
                             human health      carcinogenic effects             9.027851e-11  
                                               non-carcinogenic effects         3.409930e-10  
                                               respiratory effects, inorganics  5.486814e-11  
                             ecosystem quality freshwater ecotoxicity           1.590428e-03  
                                               freshwater eutrophication        4.519336e-08  
                             resources         dissipated water                 3.853415e-04  
                                               fossils                          1.065016e-02  
                                               land use                         5.907572e-03  
                                               minerals and metals              4.161657e-08

## Threshold 5%

In [311]:
# Replace parameters in equation
eq_chi_05 = eq.subs(par_dict)
repl = {D: D_val}
eq_chi_05 = eq_chi_05.subs(repl)

chi_dict_05 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group1_05:
        eq_chi_05_is = eq_chi_05.subs(is_dict)
        eq_chi_05_is = ratsimp(eq_chi_05_is)
              
        chi4 = eq_chi_05_is.args[0] 
        num =fraction(eq_chi_05_is.args[1])[0]
        
        chi1 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p*W_d]
        chi2 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p]
        chi3 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[W_d]
              
        chi_dict_05[method] = {'chi1': chi1, 'chi2': chi2,
                                    'chi3': chi3, 'chi4': chi4}
           

In [312]:
# Show example of the formula
eq_chi_05_is

4.16165746936438e-8 + 1.0*(4.54750575028257e-11*SR_p*W_d + 2.54150619967528e-7*SR_p + 1.26319604174516e-8*W_d)/(P_ne*SR_p)

In [313]:
chi_df_05 = pd.DataFrame.from_dict(chi_dict_05, orient='index')
chi_df_05 = chi_df_05.astype(float)
chi_df_05

chi1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             5.306783e-13   
                                               non-carcinogenic effects         1.470655e-12   
                                               respiratory effects, inorganics  1.709576e-13   
                             ecosystem quality freshwater ecotoxicity           1.581887e-05   
                                               freshwater eutrophication        1.082846e-10   
                             resources         dissipated water                 6.909440e-07   
                                               land use                         1.055452e-05   
                                               minerals and metals              4.547506e-11   

                                                                                        chi2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             5.200986e-10   
                                               non-carcinogenic effects         1.864255e-09   
                                               respiratory effects, inorganics  6.834463e-10   
                             ecosystem quality freshwater ecotoxicity           1.086016e-02   
                                               freshwater eutrophication        1.975588e-07   
                             resources         dissipated water                 1.671556e-03   
                                               land use                         3.225178e-02   
                                               minerals and metals              2.541506e-07   

                                                                                        chi3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             1.474106e-10   
                                               non-carcinogenic effects         4.085153e-10   
                                               respiratory effects, inorganics  4.748821e-11   
                             ecosystem quality freshwater ecotoxicity           4.394131e-03   
                                               freshwater eutrophication        3.007905e-08   
                             resources         dissipated water                 1.919289e-04   
                                               land use                         2.931812e-03   
                                               minerals and metals              1.263196e-08   

                                                                                        chi4  
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             9.027851e-11  
                                               non-carcinogenic effects         3.409930e-10  
                                               respiratory effects, inorganics  5.486814e-11  
                             ecosystem quality freshwater ecotoxicity           1.590428e-03  
                                               freshwater eutrophication        4.519336e-08  
                             resources         dissipated water                 3.853415e-04  
                                               land use                         5.907572e-03  
                                               minerals and metals              4.161657e-08

# Compute delta

## Threshold = 15%

In [314]:
# Replace parameters in equation
eq_delta_15 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        W_d: W_d_val}
eq_delta_15 = eq_delta_15.subs(repl)

delta_dict_15 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group2_15:
        eq_delta_15_is = eq_delta_15.subs(is_dict)
        eq_delta_15_is = ratsimp(eq_delta_15_is)
              
        delta3 = eq_delta_15_is.args[0] 
        
        num  = fraction(eq_delta_15_is.args[1])[0]
        delta1 = collect(num, [D], evaluate=False)[D]
        delta2 = collect(num, [D], evaluate=False)[1]
                
        delta_dict_15[method] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3}

In [315]:
# Show example of the formula
eq_delta_15_is

9.59210575558912e-6 + 1.0*(4.64222013454166e-7*D + 0.00201792516208315)/P_ne

In [316]:
delta_df_15 = pd.DataFrame.from_dict(delta_dict_15, orient='index', dtype="float64")
delta_df_15

delta1  \
ILCD 2.0 2018 midpoint no LT ecosystem quality marine eutrophication       4.237590e-08   
                                               terrestrial eutrophication  4.642220e-07   

                                                                             delta2  \
ILCD 2.0 2018 midpoint no LT ecosystem quality marine eutrophication       0.000185   
                                               terrestrial eutrophication  0.002018   

                                                                             delta3  
ILCD 2.0 2018 midpoint no LT ecosystem quality marine eutrophication       0.000001  
                                               terrestrial eutrophication  0.000010

## Threshold = 10%

In [317]:
# Replace parameters in equation
eq_delta_10 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        W_d: W_d_val}
eq_delta_10 = eq_delta_10.subs(repl)

delta_dict_10 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group2_10:
        eq_delta_10_is = eq_delta_10.subs(is_dict)
        eq_delta_10_is = ratsimp(eq_delta_10_is)
              
        delta3 = eq_delta_10_is.args[0] 
        
        num  = fraction(eq_delta_10_is.args[1])[0]
        delta1 = collect(num, [D], evaluate=False)[D]
        delta2 = collect(num, [D], evaluate=False)[1]
                
        delta_dict_10[method] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3}

In [318]:
# Show example of the formula
eq_delta_10_is

9.59210575558912e-6 + 1.0*(4.64222013454166e-7*D + 0.00201792516208315)/P_ne

In [319]:
delta_df_10 = pd.DataFrame.from_dict(delta_dict_10, orient='index')
delta_df_10 = delta_df_10.astype("float64")
delta_df_10

delta1  \
ILCD 2.0 2018 midpoint no LT human health      ionising radiation                        3.956410e-07   
                                               ozone layer depletion                     1.464713e-12   
                                               photochemical ozone creation              1.214914e-07   
                             ecosystem quality freshwater and terrestrial acidification  9.490167e-08   
                                               marine eutrophication                     4.237590e-08   
                                               terrestrial eutrophication                4.642220e-07   

                                                                                               delta2  \
ILCD 2.0 2018 midpoint no LT human health      ionising radiation                        2.141081e-03   
                                               ozone layer depletion                     7.056892e-09   
                                               photochemical ozone creation              5.652251e-04   
                             ecosystem quality freshwater and terrestrial acidification  4.787819e-04   
                                               marine eutrophication                     1.851849e-04   
                                               terrestrial eutrophication                2.017925e-03   

                                                                                               delta3  
ILCD 2.0 2018 midpoint no LT human health      ionising radiation                        2.498490e-05  
                                               ozone layer depletion                     1.328836e-09  
                                               photochemical ozone creation              3.303788e-06  
                             ecosystem quality freshwater and terrestrial acidification  5.371853e-06  
                                               marine eutrophication                     1.008642e-06  
                                               terrestrial eutrophication                9.592106e-06

## Threshold = 5%

In [320]:
# Replace parameters in equation
eq_delta_05 = eq.subs(par_dict)

delta_dict_05 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group1_05:
        eq_delta_05_is = eq_delta_05.subs(is_dict)
        eq_delta_05_is = ratsimp(eq_delta_05_is)
              
        delta6 = eq_delta_05_is.args[0] 
        
        num = fraction(eq_delta_05_is.args[1])[0]
        delta1 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[D*SR_p*W_d]
        delta2 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[D*W_d]
        delta3 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p*W_d]
        delta3 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p]
        delta4 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[W_d]
        
        delta_dict_05[method] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5, 'delta6': delta6}

In [321]:
# Show example of the formula
eq_delta_05_is

4.16165746936438e-8 + 1.0*(3.84072238091035e-16*D*SR_p*W_d + 1.06686732803064e-13*D*W_d + 4.2210443479052e-11*SR_p*W_d + 2.54150619967528e-7*SR_p + 1.17251231886255e-8*W_d)/(P_ne*SR_p)

In [322]:
delta_df_05 = pd.DataFrame.from_dict(delta_dict_05, orient='index')
delta_df_05 = delta_df_05.astype(float)
delta_df_05

delta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             6.365366e-19   
                                               non-carcinogenic effects         9.214040e-18   
                                               respiratory effects, inorganics  7.477412e-18   
                             ecosystem quality freshwater ecotoxicity           6.937011e-11   
                                               freshwater eutrophication        6.533736e-16   
                             resources         dissipated water                 7.482288e-12   
                                               land use                         2.335556e-10   
                                               minerals and metals              3.840722e-16   

                                                                                      delta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             1.768157e-16   
                                               non-carcinogenic effects         2.559456e-15   
                                               respiratory effects, inorganics  2.077059e-15   
                             ecosystem quality freshwater ecotoxicity           1.926948e-08   
                                               freshwater eutrophication        1.814927e-13   
                             resources         dissipated water                 2.078413e-09   
                                               land use                         6.487657e-08   
                                               minerals and metals              1.066867e-13   

                                                                                      delta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             5.200986e-10   
                                               non-carcinogenic effects         1.864255e-09   
                                               respiratory effects, inorganics  6.834463e-10   
                             ecosystem quality freshwater ecotoxicity           1.086016e-02   
                                               freshwater eutrophication        1.975588e-07   
                             resources         dissipated water                 1.671556e-03   
                                               land use                         3.225178e-02   
                                               minerals and metals              2.541506e-07   

                                                                                      delta4  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             1.459077e-10   
                                               non-carcinogenic effects         3.867599e-10   
                                               respiratory effects, inorganics  2.983321e-11   
                             ecosystem quality freshwater ecotoxicity           4.230341e-03   
                                               freshwater eutrophication        2.853636e-08   
                             resources         dissipated water                 1.742624e-04   
                                               land use                         2.380361e-03   
                                               minerals and metals              1.172512e-08   

                                                                                      delta5  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects             1.459077e-10   
                                               non-carcinogenic effects         1.459077e-10   
                                               respiratory effects, inorganics  1.459077e-10   
                             ecosystem quality freshwater ecotoxicity           1.459077e-10   
                                               freshwater eutrophication        1.459077e-10   
                             resources         dis

# Save to excel

In [323]:
if success_rate == True:
    if exploration == True:
        file_path = os.path.join(path, 'generated_files\Simplified models coefficients enhanced - symbolic - thresholds.xlsx')
    elif exploration == False:
                file_path = os.path.join(path, 'generated_files\Simplified models coefficients enhanced - symbolic - thresholds - NO EXPL.xlsx')

    with pd.ExcelWriter(file_path) as writer:
        chi_df_05.to_excel(writer, sheet_name='chi_5%')
        chi_df_10.to_excel(writer, sheet_name='chi_10%')
        chi_df_15.to_excel(writer, sheet_name='chi_15%')
        chi_df_20.to_excel(writer, sheet_name='chi_20%')    
        delta_df_05.to_excel(writer, sheet_name='delta_5%')
        delta_df_10.to_excel(writer, sheet_name='delta_10%')
        delta_df_15.to_excel(writer, sheet_name='delta_15%')
    

# Other save

In [324]:
acronyms_dict = {
    'climate change total' : 'CC',
    'carcinogenic effects' : 'HT-c',
    'ionising radiation' : 'IR' ,
    'non-carcinogenic effects' : 'HT-nc',
    'ozone layer depletion' : 'OD',
    'photochemical ozone creation': 'POC',
    'respiratory effects, inorganics': 'PM/RI',
    'freshwater and terrestrial acidification':'A',
    'freshwater ecotoxicity':'ETf',
    'freshwater eutrophication':'Ef',
    'marine eutrophication':'Em',
    'terrestrial eutrophication':'Et',
    'dissipated water':'RUw',
    'fossils':'RUe',
    'land use':'LU',
    'minerals and metals': 'RUm'}

In [325]:
chi_dict_15 = chi_df_15.to_dict(orient="index")
chi_dict_15_ac=dict((acronyms_dict[k[2]], v) for (k, v) in chi_dict_15.items())
chi_df_15_ac = pd.DataFrame.from_dict(chi_dict_15_ac, orient="index")
chi_df_15_ac


,chi1,chi2
CC,1.078929e-01,7.864316e-04
HT-c,9.040434e-09,9.027851e-11
IR,5.504030e-03,2.498490e-05
HT-nc,2.547644e-08,3.409930e-10
OD,1.950695e-08,1.328836e-09
POC,1.597902e-03,3.303788e-06
PM/RI,3.428265e-09,5.486814e-11
A,1.285446e-03,5.371853e-06
ETf,2.648409e-01,1.590428e-03
Ef,1.936128e-06,4.519336e-08


In [326]:
delta_dict_15_ac=dict((acronyms_dict[k[2]], v) for (k, v) in delta_dict_15.items())
delta_df_15_ac = pd.DataFrame.from_dict(delta_dict_15_ac, orient="index")
delta_df_15_ac

,delta1,delta2,delta3
Em,4.23759043618309e-8,0.000185184885779813,1.00864168156047e-6
Et,4.64222013454166e-7,0.00201792516208315,9.59210575558912e-6


In [327]:
if exploration == True:
    if success_rate == False:
        file_name = 'Simplified enhanced coefficients acronyms - NO SR.xlsx'
    elif success_rate == True:
        file_name = 'Simplified enhanced coefficients acronyms.xlsx'

    with pd.ExcelWriter(os.path.join(path, file_name)) as writer:
        chi_df_15_ac.to_excel(writer, sheet_name='chi_15%')
        delta_df_15_ac.to_excel(writer, sheet_name='delta_15%')